In [1]:
from pynq import Overlay
import time
import matplotlib.pyplot as plt
import numpy as np
import shutil
import os

ol = Overlay("/home/xilinx/please_work/working.bit")

In [2]:
ol?

In [4]:
oux = ol.u_x.channel1
ouy = ol.u_y.channel1
orho_u2 = ol.rho_u2.channel1
# ireset_lbm_step = ol.rst_step.channel1
ireset = ol.reset_lbm.channel1
istep = ol.step.channel1
istart_transmit = ol.start_transmit.channel1
ostep_countn = ol.step_countn.channel1
# iparam_set = ol.params.channel1
#iimg = ol.img_thing.channel1
size = 50

In [ ]:
def write_param(param, val):
    # Ensure value fits in 16 bits
    val_16bit = val & 0xFFFF  
    
    # Create 32-bit word: [16-bit index | 16-bit value]
    if param == "omega":
        gpio_value = (0 << 16) | val_16bit
    elif param == "c0":
        gpio_value = (1 << 16) | val_16bit
    elif param == "cn":
        gpio_value = (2 << 16) | val_16bit
    elif param == "cne":
        gpio_value = (3 << 16) | val_16bit
    elif param == "ce":
        gpio_value = (4 << 16) | val_16bit
    elif param == "cse":
        gpio_value = (5 << 16) | val_16bit
    elif param == "cs":
        gpio_value = (6 << 16) | val_16bit
    elif param == "csw":
        gpio_value = (7 << 16) | val_16bit
    elif param == "cw":
        gpio_value = (8 << 16) | val_16bit
    elif param == "cnw":
        gpio_value = (9 << 16) | val_16bit
    else:
        raise ValueError(f"Unknown parameter: {param}")
    
    iparam_set.write(gpio_value,-1)

def write_reset(i):
    ireset_lbm_step.write(i<<16,0x10000)

def write_step(i):
    ireset_lbm_step.write(i,0xFFFF)

# def write_img_done(i): # set MSB 1 to send image
#     iimg.write(i<<31,1<<31)

# def write_img_addr(i):
#     iimg.write(i<<16,0x7FFF0000)
    
# def write_img_data(i):
#     iimg.write(i,0x0000FFFF)

In [ ]:
import socket
import numpy as np
from pynq import Overlay
from PIL import Image

HOST = '192.168.2.99'
PORT = 9006

# Load bitstream and setup AXI interface
ol = Overlay("/home/xilinx/please_work/working.bit")
AXI = ol.img_thing.channel1

# Expecting 313 bytes (2500 bits, padded to next full byte)
EXPECTED_BYTES = 313

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    print(f"Listening on {HOST}:{PORT}...")

    conn, addr = s.accept()
    print(f"Connection from {addr}")

    with conn:
        data_buff = bytearray()
        while len(data_buff) < EXPECTED_BYTES:
            packet = conn.recv(EXPECTED_BYTES - len(data_buff))
            if not packet:
                break
            data_buff.extend(packet)

        print(f"Received {len(data_buff)} bytes")

        # Unpack bits (only first 2500 bits matter)
        bit_array = np.unpackbits(np.frombuffer(data_buff, dtype=np.uint8))[:2500]
        
        reversed_array = np.flip(bit_array)
        # Send to AXI in 16-bit chunks
        for i in range(0, 2500, 16):
            word_bits = np.flip(bit_array[i:i+16])
            if len(word_bits) < 16:
                word_bits = np.pad(word_bits, (0, 16 - len(word_bits)))  # pad last word
            word = int("".join(map(str, word_bits)), 2)

            address = i  // 16 # word address
            full_word = (1 << 31) | (address << 16) | word
            AXI.write(full_word, 0xFFFFFFFF)
            print(f"Sent word {address} to AXI")

        # Reconstruct and save image
        img_array = bit_array.reshape((50, 50)) * 255  # Convert to 0-255
        img = Image.fromarray(img_array.astype(np.uint8), mode='L')
        img.save("/home/xilinx/received_image.png")
        print("Saved image as /home/xilinx/received_image.png")

conn.close()
s.close()


In [ ]:
# import socket

# HOST = '192.168.2.99'
# PORT = 9005

In [ ]:
# AXI = ol.img_thing.channel1

# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#     s.bind((HOST, PORT))
#     s.listen()
#     conn, addr = s.accept()
#     with conn:
#         data_buff = bytearray(conn.recv(156))  
#         for i in range(0, len(data_buff), 4):
#             word_bytes = data_buff[i:i+4]
#             if len(word_bytes) < 4:
#                 continue  # skip incomplete word
#             word = int.from_bytes(word_bytes, byteorder='big') 
#             AXI.write(156 ,(1 << 31) | (i << 16) | word)
#             print("sent")
# conn.close()
# s.close()

In [ ]:
write_param("omega", 0x3000)
write_param("c0"  , 0x0E02 )
write_param("cn"  , 0x0381 )
write_param("cs"  , 0x0381 )
write_param("ce"  , 0x04BB )
write_param("cw"  , 0x0298 )
write_param("cne" , 0x012F )
write_param("cse" , 0x012F )
write_param("cnw" , 0x00A6 )
write_param("csw" , 0x00A6 )

# # make a circle
# binstr = ""
# for i in range(size):
#     for j in range(size):
#         if((i-size//2)**2 + (j-size//2)**2 < size//3**2):
#             binstr += "1";
#         else:
#             binstr += "0";
# # convert to 16 bit slices
# for sect in range(0,len(binstr),16):
#     write_img_addr(sect)
#     write_img_data(int(binstr[sect:sect+16],2))
# write_img_done(1);
            

for i in range(2):
    # reset all
    write_reset(0)
    write_reset(1) #reset
    time.sleep(0.1) # wait
    print(f"reset done") # check non-reset

    # check start transmit
    istart_transmit.write(0,-1)
    print(f"transmit state: {istart_transmit.read()}")
    write_step(0)
    time.sleep(0.1)
    # check step
    print(f"current step (expect 0): {ostep_countn.read()}")

In [6]:
# Clear images folder
size = 50
# os.system("rm imgs/*")
# os.system("rm txt/*")
ireset.write(0,-1);
ireset.write(1,-1);
times = []
for step in range(0, 100, 20):
    istep.write(step, -1)
    if(step > 0):
        u2s = []
    #     print(f"step count {ostep_countn.read()}")
        istart_transmit.write(0x8000, 0x8000)  # raise query flag
        
        t1 = time.time()
        for pixel in range(size ** 2):
            istart_transmit.write(pixel, 0x0FFF)
    #         print(orho_u2.read() & 0xFFFF, orho_u2.read() >> 16)
            u2s.append(float(orho_u2.read() >> 16) / (2**13))
        istart_transmit.write(0, 0x8000)
        times.append(time.time()-t1)
#         # Save image
#         img = np.reshape(u2s, (size, size))
#         plt.imsave(f"imgs/step{step}.png", img, vmin=0, vmax=0.1)

#         # Save text file
#         with open(f"txt/step{step}.txt", "w") as f:
#             for row in img:
#                 f.write(" ".join(f"{val:6.3f}" for val in row) + "\n")

        
print(f"avg: {np.mean(times)}")
print("done")

# # Zip the imgs folder
# shutil.make_archive("fixed_pls", 'zip', "imgs")
# shutil.make_archive("fixed_please_txt", 'zip', "txt")

avg: 0.19176316261291504
done


In [ ]:
with open("binout_lil.bytes", "wb") as f:
    for step in range(0,100):
        istep.write(step,0x00FF)
#         print(f"solver current step : {ostep_countn.read()}")
        istart_transmit.write(0x8000,0x8000) # raise query state flag
        for pixel in range(2500):
            istart_transmit.write(pixel,0x0FFF) # get pixel
            for val in [orho_u2.read()>>16, orho_u2.read()&0xFFFF, oux.read(), ouy.read()]:
                f.write((val).to_bytes(2, byteorder="little"))
        istart_transmit.write(0,0x8000) # lower query state flag

# TCP Test

In [ ]:
welcome_socket.close()
connection_socket.close()

In [ ]:
server_port = 9005;
welcome_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
welcome_socket.bind(('0.0.0.0', server_port))
welcome_socket.listen()
connection_socket, caddr = welcome_socket.accept()

In [ ]:
for step in range(0, int(1e5),50):
    write_step(step)
    frame = bytearray()

    istart_transmit.write(0x8000, 0x8000)  # raise query flag
    
    for pixel in range(size**2):
        istart_transmit.write(pixel,0x0FFF) # get pixel
        for val in [orho_u2.read()>>16, orho_u2.read()&0xFFFF, oux.read() & 0xFFFF, ouy.read() & 0xFFFF]:
            frame.extend((val).to_bytes(2, byteorder="little"))
#             print(val)
        
    istart_transmit.write(0, 0x8000)
    connection_socket.sendall(frame)
connection_socket.close()
print("done")


# Cython Server

In [ ]:
%load_ext cython

In [ ]:
%%cython -a

from pynq import Overlay
import time
import matplotlib.pyplot as plt
import numpy as np
import socket

ol = Overlay("/home/xilinx/please_work/working.bit")

size=50
oux = ol.u_x.channel1
ouy = ol.u_y.channel1
orho_u2 = ol.rho_u2.channel1
ireset_lbm_step = ol.rst_step.channel1
istart_transmit = ol.start_transmit.channel1
ostep_countn = ol.step_countn.channel1
iparam_set = ol.params.channel1

def write_param(param, val):
    # Ensure value fits in 16 bits
    val_16bit = val & 0xFFFF  
    
    # Create 32-bit word: [16-bit index | 16-bit value]
    if param == "omega":
        gpio_value = (0 << 16) | val_16bit
    elif param == "c0":
        gpio_value = (1 << 16) | val_16bit
    elif param == "cn":
        gpio_value = (2 << 16) | val_16bit
    elif param == "cne":
        gpio_value = (3 << 16) | val_16bit
    elif param == "ce":
        gpio_value = (4 << 16) | val_16bit
    elif param == "cse":
        gpio_value = (5 << 16) | val_16bit
    elif param == "cs":
        gpio_value = (6 << 16) | val_16bit
    elif param == "csw":
        gpio_value = (7 << 16) | val_16bit
    elif param == "cw":
        gpio_value = (8 << 16) | val_16bit
    elif param == "cnw":
        gpio_value = (9 << 16) | val_16bit
    else:
        raise ValueError(f"Unknown parameter: {param}")
    
    iparam_set.write(gpio_value,-1)

def write_reset(i):
    ireset_lbm_step.write(i<<16,0x10000)

def write_step(i):
    ireset_lbm_step.write(i,0xFFFF)
   

write_param("omega", 0x3500)

write_param("c0"  , 0x0E02 )
write_param("cn"  , 0x0381 )
write_param("cs"  , 0x0381 )
write_param("ce"  , 0x04BB )
write_param("cw"  , 0x0298 )
write_param("cne" , 0x012F )
write_param("cse" , 0x012F )
write_param("cnw" , 0x00A6 )
write_param("csw" , 0x00A6 )

for i in range(2):
    # reset all
    write_reset(0)
    write_reset(1) #reset
    time.sleep(0.1) # wait
    print(f"reset done") # check non-reset

    # check start transmit
    istart_transmit.write(0,-1)
    print(f"transmit state: {istart_transmit.read()}")
    write_step(0)
    time.sleep(0.1)
    # check step
    print(f"current step (expect 0): {ostep_countn.read()}")
    
server_port = 12345;
welcome_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
welcome_socket.bind(('0.0.0.0', server_port))
welcome_socket.listen()
connection_socket, caddr = welcome_socket.accept()

for step in range(0, int(1e6),100):
    write_step(step)
    frame = bytearray()

    istart_transmit.write(0x8000, 0x8000)  # raise query flag
    
    for pixel in range(size**2):
        istart_transmit.write(pixel,0x0FFF) # get pixel
        for val in [orho_u2.read()>>16, orho_u2.read()&0xFFFF, oux.read() & 0xFFFF, ouy.read() & 0xFFFF]:
            frame.extend((val).to_bytes(2, byteorder="little"))
            
    istart_transmit.write(0, 0x8000)
    connection_socket.sendall(frame)
connection_socket.close()
print("done")


In [ ]:
%%cython -a

from pynq import Overlay
import time
import matplotlib.pyplot as plt
import numpy as np
import socket

ol = Overlay("/home/xilinx/please_work/working.bit")

size=50
oux = ol.u_x.channel1
ouy = ol.u_y.channel1
orho_u2 = ol.rho_u2.channel1
ireset_lbm_step = ol.rst_step.channel1
istart_transmit = ol.start_transmit.channel1
ostep_countn = ol.step_countn.channel1
iparam_set = ol.params.channel1

def write_param(param, val):
    # Ensure value fits in 16 bits
    val_16bit = val & 0xFFFF  
    
    # Create 32-bit word: [16-bit index | 16-bit value]
    if param == "omega":
        gpio_value = (0 << 16) | val_16bit
    elif param == "c0":
        gpio_value = (1 << 16) | val_16bit
    elif param == "cn":
        gpio_value = (2 << 16) | val_16bit
    elif param == "cne":
        gpio_value = (3 << 16) | val_16bit
    elif param == "ce":
        gpio_value = (4 << 16) | val_16bit
    elif param == "cse":
        gpio_value = (5 << 16) | val_16bit
    elif param == "cs":
        gpio_value = (6 << 16) | val_16bit
    elif param == "csw":
        gpio_value = (7 << 16) | val_16bit
    elif param == "cw":
        gpio_value = (8 << 16) | val_16bit
    elif param == "cnw":
        gpio_value = (9 << 16) | val_16bit
    else:
        raise ValueError(f"Unknown parameter: {param}")
    
    iparam_set.write(gpio_value,-1)

def write_reset(i):
    ireset_lbm_step.write(i<<16,0x10000)

def write_step(i):
    ireset_lbm_step.write(i,0xFFFF)
   

write_param("omega", 0x3500)

write_param("c0"  , 0x0E02 )
write_param("cn"  , 0x0381 )
write_param("cs"  , 0x0381 )
write_param("ce"  , 0x04BB )
write_param("cw"  , 0x0298 )
write_param("cne" , 0x012F )
write_param("cse" , 0x012F )
write_param("cnw" , 0x00A6 )
write_param("csw" , 0x00A6 )

for i in range(2):
    # reset all
    write_reset(0)
    write_reset(1) #reset
    time.sleep(0.1) # wait
    print(f"reset done") # check non-reset

    # check start transmit
    istart_transmit.write(0,-1)
    print(f"transmit state: {istart_transmit.read()}")
    write_step(0)
    time.sleep(0.1)
    # check step
    print(f"current step (expect 0): {ostep_countn.read()}")
    
# --------------------------------------------------------------------------------------------------------------------------#
# --------------------------------------------------------------------------------------------------------------------------#

cdef int pixel, idx, step
cdef unsigned short val0, val1, val2, val3
    
server_port = 12345;
welcome_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
welcome_socket.bind(('0.0.0.0', server_port))
welcome_socket.listen()
connection_socket, caddr = welcome_socket.accept()
connection_socket.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)

num_pixels = size * size

data_array = np.zeros((num_pixels, 4), dtype=np.uint16)

for step in range(0, int(1e6), 100):
    write_step(step)
    istart_transmit.write(0x8000, 0x8000)  # raise query flag
    
    for pixel in range(num_pixels):
        istart_transmit.write(pixel,0x0FFF)

        val0 = (orho_u2.read() >> 16) & 0xFFFF
        val1 = orho_u2.read() & 0xFFFF
        val2 = oux.read() & 0xFFFF
        val3 = ouy.read() & 0xFFFF

        data_array[pixel, 0] = val0
        data_array[pixel, 1] = val1
        data_array[pixel, 2] = val2
        data_array[pixel, 3] = val3

    istart_transmit.write(0, 0x8000)
    byte_frame = data_array.tobytes()
    connection_socket.sendall(byte_frame)
    
connection_socket.close()
print("done")


# dma test

In [ ]:
%load_ext cython

In [ ]:
from pynq import Overlay
from pynq import allocate
import time
import matplotlib.pyplot as plt
import numpy as np
import shutil
import os
import socket

ol = Overlay("/home/xilinx/please_work/working.bit")

size=50
cellCount = size * size

In [ ]:
# %%cython -a
os.system("rm imgs/*")
os.system("rm thanks.zip")
ostep_countn = ol.step_countn.channel1
ireset_lbm_step = ol.rst_step.channel1
iparam_set = ol.params.channel1
iimg = ol.img_thing.channel1
def write_img_done(i): # set MSB 1 to send image
    iimg.write(i<<31,1<<31)

def write_img_addr(i):
    iimg.write(i<<16,0x7FFF0000)
    
def write_img_data(i):
    iimg.write(i,0x0000FFFF)
# make a circle
binstr = ""
for i in range(size):
    for j in range(size):
#         if((i-size//2)**2 + (j-size//2)**2 < (size//3)**2):
#             binstr += "1";
#         else:
#             binstr += "0";
            if( 20 < i < 22 and 20 < j < 30):
                binstr += "1"
            else:
                binstr += "0"
            

param_map = {
    "omega": 0,
    "c0": 1,
    "cn": 2,
    "cne": 3,
    "ce": 4,
    "cse": 5,
    "cs": 6,
    "csw": 7,
    "cw": 8,
    "cnw": 9,
}

def write_param(param, val):
    # Ensure value fits in 16 bits
    val_16bit = val & 0xFFFF  
    param_index = param_map[param]

    gpio_value = (param_index << 16) | val_16bit
    iparam_set.write(gpio_value,-1)

def write_reset(i):
    ireset_lbm_step.write(i<<16,0x10000)

def write_step(i):
    ireset_lbm_step.write(i,0xFFFF)
    
write_param("omega", 0x3500)
write_param("c0"  , 0x0E02 )
write_param("cn"  , 0x0381 )
write_param("cs"  , 0x0381 )
write_param("ce"  , 0x04BB )
write_param("cw"  , 0x0298 )
write_param("cne" , 0x012F )
write_param("cse" , 0x012F )
write_param("cnw" , 0x00A6 )
write_param("csw" , 0x00A6 )

#--------------------------------------------------------------------------------------------------------------------------#

for i in range(2):
    # reset all
    write_reset(0)
    write_reset(1) #reset
    time.sleep(0.1) # wait

    write_step(0)
    time.sleep(0.1)
    # check step
print(f"current step (expect 0): {ostep_countn.read()}")

# convert to 16 bit slices
for sect in range(0,len(binstr),16):
    write_img_addr(sect//16)
    write_img_data(int(binstr[sect:sect+16],2))
write_img_done(1);
# write_img_done(0);

    
#--------------------------------------------------------------------------------------------------------------------------#
dma_o = ol.axi_dma
dma = dma_o.recvchannel


# server_port = 12345;
# welcome_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# welcome_socket.bind(('0.0.0.0', server_port))
# welcome_socket.listen()
# connection_socket, caddr = welcome_socket.accept()
# connection_socket.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)

print(f"""    DMA data: 
      running: {dma.running}, idle: {dma.idle}, error: {dma.error}""")

# print(dma_o.register_map)

times = []
for step in range(1, int(300), 30):
    frame_buffer = allocate(shape=(cellCount * 4+10,), dtype=np.int16) # buffer needs to be overallocated
    write_step(step)
    dma.start()
    cstep = ostep_countn.read()
    print(f"current step: {cstep}")
    t1 = time.time()
    dma.transfer(frame_buffer) 
    dma.wait() 
    times.append((time.time()-t1))
    
    rhos = frame_buffer[3::4]
    rhos = [float(x)/2**13 for x in rhos[0:cellCount]]
    
    img = np.reshape(rhos, (size, size))
#     print(img)
    plt.imsave(f"imgs/step{cstep}.png", img)

#     connection_socket.sendall(frame_buffer.tobytes())
#     print(frame_buffer)
    
    frame_buffer.invalidate()
    frame_buffer.flush()
    frame_buffer.freebuffer()
    dma.stop()
    del frame_buffer
# connection_socket.close()
print(f"avg time: {np.mean(times)}")

# Zip the imgs folder
shutil.make_archive("thanks", 'zip', "imgs")
print("done")

In [ ]:
dma_o.register_map

In [ ]:
# manual reset of DMA for debug
dma_o.register_map.S2MM_DMACR.Reset = 1
dma.start()

# unused tests

In [ ]:
# test to see if there are any changes in outputs
prev_rhos = []
for step in range(0,100):
#     iinterface_reset.write(0,1)
    istep.write(step,0x00FF)

    rhos = []

    istart_transmit.write(0x8000,0x8000) # raise query state flag
    for pixel in range(2500):
        istart_transmit.write(pixel,0x0FFF) # get pixel
        rhos.append(oux.read())

    istart_transmit.write(0,-1)
#     iinterface_reset.write(1,1)
    
    if(prev_rhos == []):
        prev_rhos = rhos.copy()
    else:
        print(f"solver current step : {ostep_countn.read()}, rhos equal? {prev_rhos == rhos}")
    
print(f"ux at step 1, pixel 70: {oldux.read() & 0xFFFF}, ux at step 20, pixel 70: {oldux.read() >> 13}") 